In [1]:
#project Predict Low Review Score
import pandas as pd 
df = pd.read_csv("C:/Users/H896994/Desktop/Projects/Olist_project/cleaned_olist_master.csv")
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,revenue,month,delivery_time,is_late,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,4.0,500.0,19.0,8.0,13.0,38.71,2017-10,8.0,False,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,4.0,500.0,19.0,8.0,13.0,38.71,2017-10,8.0,False,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,4.0,500.0,19.0,8.0,13.0,38.71,2017-10,8.0,False,housewares
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,...,4.0,500.0,19.0,8.0,13.0,37.77,2017-08,2.0,False,housewares
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,...,4.0,500.0,19.0,8.0,13.0,37.77,2017-08,5.0,False,housewares


In [4]:
#Create target variable 
#1 = bad review
#0 = Good review

df['low_review'] = df['review_score'].apply(lambda x: 1 if x<=2 else 0)
df['low_review'].value_counts()

low_review
0    97845
1    17014
Name: count, dtype: int64

In [5]:
features = ['price','freight_value','delivery_time']
x= df[features]
y= df['low_review']
x.head()

,price,freight_value,delivery_time
0,29.99,8.72,8.0
1,29.99,8.72,8.0
2,29.99,8.72,8.0
3,29.99,7.78,2.0
4,29.99,7.78,5.0


In [6]:
x.isnull().sum()

price            0
freight_value    0
delivery_time    8
dtype: int64

In [7]:
x= x.dropna()
y =y[x.index]

In [9]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size =0.2,random_state = 42)
print("Train Size",x_train.shape)
print("Test Size", x_test.shape)

Train Size (91880, 3)
Test Size (22971, 3)


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,roc_auc_score

rf = RandomForestClassifier(
    n_estimators = 200,
    class_weight ='balanced',
    random_state = 42)
rf.fit(x_train, y_train)
y_probs_rf = rf.predict_proba(x_test)[:, 1]
print("Randon Forest AUC:",roc_auc_score(y_test,y_probs_rf))

Randon Forest AUC: 0.7957067766464858


In [15]:
import joblib

model_package = {
    "model": rf,
    "features": x_train.columns.tolist()
}

joblib.dump(model_package, "C:/Users/H896994/Desktop/Projects/Olist_project/Olist-review-prediction/models/final_model.pkl")

['C:/Users/H896994/Desktop/Projects/Olist_project/Olist-review-prediction/models/final_model.pkl']

In [16]:
loaded_package = joblib.load("C:/Users/H896994/Desktop/Projects/Olist_project/Olist-review-prediction/models/final_model.pkl")

loaded_model = loaded_package["model"]
feature_order = loaded_package["features"]

loaded_model.predict(x_test[feature_order][:5])

array([0, 0, 0, 0, 1], dtype=int64)